# Creating data input spreadsheets

We want to create a workbook with:

- Instructions (extended)
- Data contributor details
- Updated trait codes and descriptions
- Updated vocabularies
- Updated species list and references
- Data entry table


## Setup

### Import modules

In [1]:
# work with paths in operating system
from pathlib import Path
import os

# work with xlsx workbooks
import openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Alignment, PatternFill, Border, Font # Side, Alignment, Protection,
from openpyxl.formatting import Rule
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.worksheet.datavalidation import DataValidation


# For database connection
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor


### Define paths for input and output

In [2]:
repodir = Path("../../") 
inputdir = repodir / "data" / "input-form"
os.listdir(inputdir)

['fireveg-trait-input-spreadsheet_V3_DE_trial.xlsx',
 '~$fireveg-trait-input-model.xlsx',
 'test.xlsx',
 '~$fireveg-trait-input-spreadsheet_V3_DE_trial.xlsx',
 'fireveg-trait-input-model.xlsx']

### Database connection

Function to parse connection parameters from a file

In [3]:
def read_dbparams(filename,section="postgresql"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


Reading the default parameters for this session:

In [4]:
filename = repodir / 'secrets' / 'database.ini'
dbparams=read_dbparams(filename,section='aws-lght-sl')

In [5]:
if "conn" not in globals() or conn.closed!=0:
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**dbparams)
if "cur" not in globals() or cur.closed:
    cur = conn.cursor(cursor_factory=DictCursor)

Connecting to the PostgreSQL database...


## Create workbook

### Styles
Define styles to be used across the workbook

In [6]:
cent_align=Alignment(horizontal='center', vertical='center', wrap_text=False)
wrap_align=Alignment(horizontal='left', vertical='top', wrap_text=True)

sheet_colors = {"instructions": "1072BA" , "entry": "10BA72", "default":"505050"}

table_style={"Instructions":TableStyleInfo(name="TableStyleMedium9", showFirstColumn=True, showLastColumn=False, 
                                           showRowStripes=True, showColumnStripes=False),
             "Contributor": TableStyleInfo(name="TableStyleMedium18", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Lists": TableStyleInfo(name="TableStyleMedium14", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Info":  TableStyleInfo(name="TableStyleMedium14", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Vocabularies": TableStyleInfo(name="TableStyleMedium14", showFirstColumn=True,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False),
             "Entry": TableStyleInfo(name="TableStyleMedium18", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=False, showColumnStripes=False)

             }




### Initialise workbook

In [7]:
wb = Workbook()

### Add sheets

Add worksheets in order, specify column dimentsions as needed and establish the tab color for each sheet.

##### Instructions

In [8]:
ws = wb.active
ws.title = "Instructions"
ws.column_dimensions["B"].width = 90
ws.column_dimensions["C"].width = 40
ws.sheet_properties.tabColor = sheet_colors["instructions"]


#### Contributor

In [9]:
ws = wb.create_sheet("Contributor")
ws.column_dimensions["A"].width = 30
ws.column_dimensions["B"].width = 60
ws.sheet_properties.tabColor = sheet_colors["entry"]

#### Data Entry

In [10]:
ws = wb.create_sheet("Data entry")
ws.sheet_properties.tabColor = sheet_colors["entry"]

for col in ("A","B","C","E","G","I","N","O"):
    ws.column_dimensions[col].width = 25
for col in ("D","F","H","J","K","L","M"):
    ws.column_dimensions[col].width = 12

#### Species lists

In [11]:
ws = wb.create_sheet("Species list")
ws.column_dimensions["A"].width = 90
ws.sheet_properties.tabColor = sheet_colors["default"]

#### References

In [12]:
ws = wb.create_sheet("References")

ws.column_dimensions["A"].width = 30
ws.column_dimensions["B"].width = 60

ws.sheet_properties.tabColor = sheet_colors["default"]

#### Trait descriptions

In [13]:
ws = wb.create_sheet("Trait description")
ws.column_dimensions["A"].width = 12
ws.column_dimensions["B"].width = 30
ws.column_dimensions["C"].width = 70
ws.sheet_properties.tabColor = sheet_colors["default"]

#### Vocabularies

In [14]:
ws = wb.create_sheet("Vocabularies")
ws.column_dimensions["A"].width = 30
ws.column_dimensions["B"].width = 60
ws.sheet_properties.tabColor = sheet_colors["default"]

### Add content to sheets

For each sheet we will add the tables and content as required.
#### Instructions

In [15]:
instructions = [
"""
Fill in your name and affilation in the "Contributor" tab, so that we can keep track of your contributions. Optionally fill in contact information for queries regarding your contribution.
""",
"""
Go to sheet "Data Entry" and fill one (or more) record(s) for each combination of reference + species + trait. Use "Insert > Table Rows Above/Below" to ensure new records have same format and validation options.
""",
"""
For each record, select references (main source and original sources columns) from the drop down list. If reference is not found, go to list of reference and add it to the table (use "Insert > Table Rows Above/Below" to add record to the list of references)
""",
"""
For each record, type in species name as given by main source in "original_species_name" column. A XLOOKUP function will look for a match in the species code table (list_spcode) and populate columns species_code and species_name, but this can be overridden with a manual entry if needed.	Go to list of species
""",
"""
Select a trait from the drop down menu. A XLOOKUP function will look at the trait code table and populate columns for trait name and trait type (categorical or numerical). The choice will determine the list of values for the "norm_value" column
""",
"""
Add raw value as given by original source, might include values, units and short explanatory text about observation or measurement
""",
"""
For numeric trait values (e.g. age in years) we use a triplet of integer values (columns best, lower and upper) to describe a fuzzy number. Fill out any needed numbers and leave other columns blank. If in doubt leave all columns blank. Examples a raw value of "5 (3-7)" would be best:5, lower:3 upper:7; a value of ">5" would be lower:5, best:blank, upper:blank; etc. This column is colored red if the selected trait is not numerical.

For categorical variables, use values from drop-down list. The list will update when a categorical trait is selected and will be colored red if the selected trait is not categorical. If raw value does not match any of the options, leave blank. Values not in the dropdown list will not be imported in the database, but you can add a comment in the "notes" column.
""",
""" 
Fill method of estimation from drop down list.
""",
"""
Add any notes, observations or comments in column "notes". Please avoid using colors or any other formatting, nor add comment on particular cells, rather write all comments as text in the "notes" column.
"""]

In [16]:
instructions[0]

'\n    Fill in your name and affilation in the "Contributor" tab, so that we can keep track of your contributions. Optionally fill in contact information for queries regarding your contribution.\n    '

In [17]:
links = [("#Contributor!A1","Go to 'Contributor' table"),
         ("#'Data entry'!A1","Go to 'Data Entry' table"),
         ("#'References'!A1","Go to 'References' table"),
         ("#'Species list'!A1","Go to 'Species list' table"),
         ("#'Trait description'!A1","Go to 'Trait description' table"),
         None,
         ("#'Vocabularies'!A1","Go to 'Vocabularies' table"),
         None,None,None,None,None,None,None]



In [18]:
ws = wb["Instructions"]
ws.append(["Step", "Instructions","Links"])
#ws.merge_cells(start_row=1, start_column=2, end_row=1, end_column=6)
for k in range(len(instructions)):
    ws.cell(k+2,1).value=k+1
    ws.cell(k+2,1).alignment=cent_align
    ws.cell(k+2,2).value=instructions[k]
    #ws.merge_cells(start_row=k+2, start_column=2, end_row=k+2, end_column=6)
    ws.cell(k+2,2).alignment=wrap_align
    if links[k] is not None:
        cell=ws.cell(k+2,3)
        cell.value=links[k][1]
        cell.hyperlink=links[k][0]
        #'=HYPERLINK("{}", "{}")'.format(links[k][0], links[k][1])
        cell.style = "Hyperlink"



Create table with a default style with striped rows and banded columns

In [19]:
tab = Table(displayName="Instructions", ref="A1:C{}".format(len(instructions)+1))
tab.tableStyleInfo = table_style["Instructions"]

Table must be added using ws.add_table() method to avoid duplicate names.
Using this method ensures table name is unque through out defined names and all other table name. 

In [20]:
ws.add_table(tab)


#### Contributor
Whoever is transcribing the data, should leave their name and affiliation and contact information for any query arising from data import.

In [21]:
ws = wb["Contributor"]

data = [
    ['Name', """ Your name """],
    ['Affiliation', """ Your institution """],
    ['Contact', """ e-mail or phone """],
    
]

ws.append(["Field", "Your response"])

for row in data:
    ws.append(row)
    
tab = Table(displayName="Contributor", ref="A1:B4")
tab.tableStyleInfo = table_style["Contributor"]
ws.add_table(tab)



#### Species List

In [22]:
#cur = conn.cursor(cursor_factory=DictCursor)
cur.execute('SELECT "scientificName","speciesCode_Synonym" FROM species.caps;') 
species = cur.fetchall()



In [23]:
ws = wb["Species list"]
ws.append(["Scientific Name","Code"])

for row in species:
    ws.append(row)
    
tab = Table(displayName="SpeciesList", ref="A1:B{}".format(ws.max_row))

tab.tableStyleInfo = table_style["Lists"]
ws.add_table(tab)



#### References

In [24]:

cur.execute("SELECT ref_code,ref_cite FROM litrev.ref_list") 
references = cur.fetchall()



In [25]:
ws = wb["References"]
ws.append(["Code", "Full reference"])

for row in references:
    ws.append(row)
  

In [26]:
#ws.max_row
for k in range(2,ws.max_row+1):
    ws.cell(k,2).alignment=wrap_align
    
tab = Table(displayName="References", ref="A1:B{}".format(ws.max_row))


In [27]:

tab.tableStyleInfo = table_style["Lists"]
ws.add_table(tab)


#### Trait descriptions

In [28]:
cur.execute("SELECT code,name,description,value_type,life_stage,life_history_process,priority FROM litrev.trait_info ORDER BY code")
trait_info = cur.fetchall()


In [29]:
for k in trait_info[1].values():
    print(k)

germ1
Seedbank Type

categorical
Seed
Germination
2nd tranche


In [30]:
ws = wb["Trait description"]
ws.append(["Trait Code", "Trait Name", "Description", "Type", "Life stage", "Life history process", "Priority"])

for row in trait_info:
    ws.append(row)
    
#ws.max_row
for k in range(2,ws.max_row+1):
    ws.cell(k,3).alignment=wrap_align
    
tab = Table(displayName="TraitInformation", ref="A1:G{}".format(ws.max_row))

tab.tableStyleInfo = table_style["Info"]
ws.add_table(tab)


#### Vocabularies

In [31]:
cur.execute("SELECT code,category_vocabulary,method_vocabulary FROM litrev.trait_info WHERE category_vocabulary IS NOT NULL ORDER BY code")
trait_vocabs = cur.fetchall()


In [32]:
trait_vocabs

[['repr2', 'postfire_response', None],
 ['surv1', 'resprouting_vocabulary', None]]

In [33]:
ws = wb["Vocabularies"]

k=1

for record in trait_vocabs:
    print(record)
    ws.cell(row=k,column=1,value="Lookup table for trait %s" % record['code'])
    k=k+1
    tab_first_row=k
    ws.cell(row=k,column=1,value="Valid values")
    ws.cell(row=k,column=2,value="Description")

    # we can add json-like comments to the vocabulary in postgres, and then read them as a dictionary
    cur.execute("SELECT pg_catalog.obj_description(t.oid, 'pg_type')::json from pg_type t where typname = %s;",(record['category_vocabulary'],))
    vocabs = cur.fetchone()
    vocab=vocabs[0]
    for key in vocab.keys():
        k=k+1
        ws.cell(row=k,column=1,value=key)
        ws.cell(row=k,column=2,value=vocab[key])
        ws.cell(row=k,column=2).alignment=wrap_align

    tab_last_row=k
    
    tab = Table(displayName="lookup_%s" % record['code'], ref="A{}:B{}".format(tab_first_row,tab_last_row))

    tab.tableStyleInfo = table_style["Vocabularies"]
    ws.add_table(tab)
    k=k+2


['repr2', 'postfire_response', None]
['surv1', 'resprouting_vocabulary', None]


#### Data entry

In [34]:
ws = wb["Data entry"]
hdr=["Main source", "Original sources", "Original species name", "Species code", "Species name", 
           "Trait code", "Trait name","Trait type","Raw value", "Norm value", 
           "Best", "Lower", "Upper", "Method of estimation","Notes"]
ws.append(hdr)


##### Data validation


In [35]:
dv_ref = DataValidation(type="list",
                    formula1="""=INDIRECT("References[Code]")""",
                    allow_blank=True)

dv_fuzzy = DataValidation(type="decimal",
                    operator="greaterThanOrEqual",
                    formula1=0)

dv_trait = DataValidation(type="list",
                    formula1="""=INDIRECT("TraitInformation[Trait Code]")""")

dv_vvalue = DataValidation(type="list",
                    formula1="""=INDIRECT(CONCATENATE("lookup_",$F2,"[Valid values]"))""")

# custom error message
dv_ref.error ='Your entry is not in the list'
dv_ref.errorTitle = 'Invalid Entry'
dv_trait.error ='Your entry is not in the list'
dv_trait.errorTitle = 'Invalid Entry'

# custom prompt message
dv_ref.prompt = 'Please select from the list of references'
dv_ref.promptTitle = 'List Selection'
dv_vvalue.prompt = """For categorical traits, please select trait first and then select one value from the dropdown list, otherwise leave blank.
For quantitative traits, leave blank and fill Best/Lower/Upper columns."""
dv_vvalue.promptTitle = 'Accepted values for trait'

# add validation ranges
dv_ref.add("A2:A21")
dv_trait.add("F2:F21")
dv_fuzzy.add("K2:M21")
dv_vvalue.add("J2:J21")

## add to sheet
ws.add_data_validation(dv_vvalue)
ws.add_data_validation(dv_ref)
ws.add_data_validation(dv_trait)
ws.add_data_validation(dv_fuzzy)

##### Lookup functions

In [36]:
for row in range(2,22):
    cell=ws.cell(row=row,column=7)
    # problem: using XLOOKUP adds a @ at the beginning of the formula
    #r"""=XLOOKUP(F2,INDIRECT("TraitInformation[Trait Code]"),INDIRECT("TraitInformation[Trait Name]"),"No match")"""
    cell.value="""=VLOOKUP($F{}, INDIRECT("TraitInformation"), 2, FALSE)""".format(cell.row)
    #r"""=XLOOKUP(F2,INDIRECT("TraitInformation[Trait Code]"),INDIRECT("TraitInformation[Type]"),"No match")"""
    cell=ws.cell(row=row,column=8)
    cell.value="""=VLOOKUP($F{}, INDIRECT("TraitInformation"), 4, FALSE)""".format(cell.row)
    cell=ws.cell(row=row,column=4)
    cell.value="""=VLOOKUP($C{}, INDIRECT("SpeciesList"), 2, FALSE)""".format(cell.row)
    cell=ws.cell(row=row,column=5)
    cell.value="""=VLOOKUP($C{}, INDIRECT("SpeciesList"), 1, FALSE)""".format(cell.row)
    #=XLOOKUP(C31,INDIRECT("list_spcode[species_name]"),INDIRECT("list_spcode[species_code]"),"No match")

##### Conditional formatting

In [37]:
red_fill = PatternFill(bgColor="FFC7CE")
dxf = DifferentialStyle(fill=red_fill)
r = Rule(type="expression", dxf=dxf, stopIfTrue=True)
r.formula = ['$H2="categorical"']
ws.conditional_formatting.add("K2:M21", r)

r2 = Rule(type="expression", dxf=dxf, stopIfTrue=True)
r2.formula = ['$H2="numerical"']
ws.conditional_formatting.add("J2:J21", r2)

##### Add Table

In [38]:
cell=ws.cell(row=21,column=len(hdr))

tab = Table(displayName="DataEntry", ref="A1:{}21".format(cell.column_letter))
tab.tableStyleInfo = table_style["Entry"]
ws.add_table(tab)


### Finalise workbook

In [39]:
wb.save(inputdir / "fireveg-trait-input-model.xlsx")

In [72]:
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
